In [8]:
!pip3 install flask-sqlalchemy

In [9]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table,Column,String
import glob
import os
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper

#SQL Alchemy setup

#Create engine that will allow us to communicate with database
engine=create_engine('sqlite:///kinase1.sqlite',echo=False)

#Creating session which is the middle ground to talk to our engine
Session=sessionmaker(bind=engine)
session=Session()

#Map which table in database will be related to each class
Base=declarative_base()

#Create a metadata instance
#A metadata is an object container that will store attributes and name of table 
metadata=MetaData(engine)

#Define structure of table
class kinase_table(object):
    def __init__(self,description,ref_des):
        self.Gene=Gene
        self.Group=Group
        self.Family=Family
        self.Subfamily=Subfamily
        self.Alias=Alias
        self.Uniprot_ID=Uniprot_ID
        self.Uniprot_Code=Uniprot_Code
        self.Protein_Name=Protein_Name
        self.Gene_Name=Gene_Name
        self.Cellular_Location=Cellular_Location
    
    def __repr__(self):
        return f'{self.Gene,self.Group,self.Family,self.Subfamily,self.Alias,self.Uniprot_ID,self.Uniprot_Code,self.Protein_Name,self.Gene_Name,self.Cellular_Location}'


In [20]:
#Declaring a table

#Defining a function that defines table, we define this function so that we can keep table names dynamic
#That is, we can have multiple tables in database and each table can have a different name
def table_definition(table_name):
    #Define table structure, here table_name varies
    #We want to make table_define available outside function so we declare it as a function attribute
    table_definition.table_define=Table(table_name,metadata,
    Column('Gene',String,),
    Column('Group',String),
    Column('Family',String),
    Column('Subfamily',String),
    Column('Alias',String),
    Column('Uniprot_ID',String,primary_key=True),
    Column('Uniprot_Code',String),
    Column('Protein_Name',String),
    Column('Gene_Name',String),
    Column('Cellular_Location',String)                                    
    )
    
    #Create table
    #Note that we used the engine from function
    metadata.create_all(engine)
    
    #Use mapper to define components of class as well as table definition together at once
    mapper(product_table,table_definition.table_define,non_primary=True)

In [89]:
#This function will create a separate table for each csv file, if you have multiple csv files
#Name of table will be extracted from file name. File name contains product name. 
#Each table will be identified by product name
# It will read each excel file in the folder and insert bom into table
def create_table(file_name):
    
    #Read file into dataframe
    csv_data=pd.read_csv(file_name)
    
    #Convert dataframe to list and store in same variable
    csv_data=csv_data.values.tolist()
            
    #Use table_definition function to define table structure
    table_definition("kinase")
        
    #Loop through list of lists, each list in create_bom_table.xls_data is a row
    for row in csv_data:
            
        #Each element in the list is an attribute for the table class
        #Iterating through rows and inserting into table
        ins=table_definition.table_define.insert().values(
        Gene=row[0],Group=row[2],Family=row[3],Subfamily=row[4],
        Alias=row[5],Uniprot_ID=row[6],Uniprot_Code=row[7],Protein_Name=row[8],Gene_Name=row[9],
        Cellular_Location=row[10])
        conn=engine.connect()
        conn.execute(ins)
      

In [90]:
#Calling function, argument is path of folder where all CSV files are stored
create_table("Kinase-try.csv")